In [13]:
import numpy as np
import pandas as pd
from utils import *
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold



In [14]:
class Grid(BaseEstimator, TransformerMixin):
    def __init__(self, width, height):
        self.width = width
        self.height = height
    
    def fit(self):
        return self
       
    def predict(self, X, y):
        return X, y
          

In [99]:
trace_size = 15

#############

all_days = data['4. close']

dataset = labeled_traces_from_array(all_days, trace_size)


In [113]:
X = np.array([trace for (trace, _) in dataset])
y = np.array([int(label >= 0) for (_, label) in dataset])

X_train, X_val, y_train, y_val = train_test_split(X, y, random_state=23, test_size=0.2, stratify=y)

In [125]:
grid =  itrac_grid.Grid(height=60, width=trace_size, trace_size=trace_size, grid_type='wins')
n_splits = 4


kf = KFold(n_splits=n_splits)

# grid.fit(X_train, y_train)

metric_sum = 0
for train_index, test_index in kf.split(X_train):
    kX_train, kX_test = X[train_index], X[test_index]
    ky_train, ky_test = y[train_index], y[test_index]
    
    grid.fit(kX_train, ky_train, eliminate_noise_thold=0.008)
    metric = grid.get_metrics(kX_test, ky_test, alpha_plus=0.7, alpha_minus=0.7)['criteria2_above']
    print(metric)
    metric_sum += metric
    
average_metric = metric_sum/n_splits

print('metric sum:', metric_sum)
print('avg metric:', average_metric)
    
    
    
        
        

0.16746411483253598
0.004926108374384119
0.07755474452554743
0.11749347258485643
metric sum: 0.367438440317324
avg metric: 0.091859610079331
